# **Contextual RAG**

Contextual Retrieval-Augmented Generation (RAG) is an advanced RAG technique that improves response relevance and efficiency by incorporating contextual compression during the retrieval process. Traditional RAG retrieves and sends full documents to the generation model, which may include irrelevant information, leading to higher costs and less accurate responses.

In Contextual RAG, the retrieved documents are processed through a Document Compressor before being passed to the language model. This compressor extracts and retains only the most relevant information for the query, or even discards entire irrelevant documents. This approach reduces the noise in the retrieved context, resulting in more precise, concise, and cost-effective responses from the generation model.

Reference: [Contextual RAG](https://python.langchain.com/docs/how_to/contextual_compression/)

## **Initial Setup**

In [1]:
!pip install --q athina chromadb


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ['ATHINA_API_KEY'] = os.getenv('ATHINA_API_KEY')

# Optional: Verify keys are loaded
if not os.environ["OPENAI_API_KEY"] or not os.environ['ATHINA_API_KEY']:
    print("Warning: API keys not loaded from .env file")

## **Indexing**

In [4]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [14]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path="./context.csv", encoding="utf-8")
docs = loader.load()


In [16]:
# load data
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path="./context.csv", encoding="utf-8")
documents = loader.load()

In [20]:
# split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [26]:
!pip install faiss-cpu



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from langchain_community.vectorstores import FAISS


In [28]:
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)


## **Retriever**

In [29]:
# create retriever
retriever = vectorstore.as_retriever()

## **Contextual Retriever**

In [30]:
# create llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [31]:
# create compression retriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [32]:
# checking compressed doc
compressed_docs = compression_retriever.invoke("what are points on a mortgage")
compressed_docs

[Document(metadata={'source': './context.csv', 'row': 1}, page_content='Discount points, also called mortgage points or simply points, are a form of pre-paid interest available in the United States when arranging a mortgage. One point equals one percent of the loan amount. By charging a borrower points, a lender effectively increases the yield on the loan above the amount of the stated interest rate. Borrowers can offer to pay a lender points as a method to reduce the interest rate on the loan, thus obtaining a lower monthly payment in exchange for this'),
 Document(metadata={'source': './context.csv', 'row': 1}, page_content="points is the concept of the 'no closing cost loan', in which the consumer accepts a higher interest rate in return for the lender paying the loan's closing costs up front. In some cases a purchaser can negotiate with the seller to get them to pay seller's points which can be used to pay mortgage points."),
 Document(metadata={'source': './context.csv', 'row': 1}

## **RAG Chain**

In [33]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the following context.
If you don't find the answer in the context, just say that you don't know.
Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": compression_retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
# response
response = rag_chain.invoke("what are points on a mortgage")
response

"Points on a mortgage are a form of pre-paid interest that can be paid by the borrower to reduce the interest rate on the loan, resulting in a lower monthly payment. Points are typically equal to one percent of the loan amount. Additionally, points can also refer to the concept of the 'no closing cost loan', where the consumer accepts a higher interest rate in exchange for the lender paying the closing costs upfront."

## **Preparing Data for Evaluation**

In [35]:
# create dataset
questions = ["what are points on a mortgage"]
response = []
contexts = []

# Inference
for query in questions:
  response.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in compression_retriever.get_relevant_documents(query)])

# To dict
data = {
    "query": questions,
    "response": response,
    "context": contexts,
}

C:\Users\patel\AppData\Local\Temp\ipykernel_18128\255348039.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([docs.page_content for docs in compression_retriever.get_relevant_documents(query)])


In [36]:
# create dataset
from datasets import Dataset
dataset = Dataset.from_dict(data)

In [37]:
# create dataframe
import pandas as pd
df = pd.DataFrame(dataset)

In [38]:
df

,query,response,context
0,what are points on a mortgage,Points on a mortgage are a form of pre-paid in...,"[Discount points, also called mortgage points ..."


In [39]:
# Convert to dictionary
df_dict = df.to_dict(orient='records')

# Convert context to list
for record in df_dict:
    if not isinstance(record.get('context'), list):
        if record.get('context') is None:
            record['context'] = []
        else:
            record['context'] = [record['context']]

## **Evaluation in Athina AI**

We will use **Context Relevancy** eval here. It Measures the relevancy of the retrieved context, calculated based on both the query and contexts. To learn more about this. Please refer to our [documentation](https://docs.athina.ai/api-reference/evals/preset-evals/overview) for further details

In [41]:
import os

# Set env variables directly
os.environ["OPENAI_API_KEY"] = "your-openai-key"
os.environ["ATHINA_API_KEY"] = "your-athina-key"

In [40]:
# set api keys for Athina evals
from athina.keys import AthinaApiKey, OpenAiApiKey
OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

c:\Users\patel\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


NameError: name 'Sandbox' is not defined

In [42]:
# load dataset
from athina.loaders import Loader
dataset = Loader().load_dict(df_dict)

In [44]:
!pip uninstall athina -y
!pip install -U athina


Found existing installation: athina 1.7.12
Uninstalling athina-1.7.12:
  Successfully uninstalled athina-1.7.12
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
      --------------------------------------- 0.1/7.2 MB 3.3 MB/s eta 0:00:03
     -- ------------------------------------- 0.5/7.2 MB 6.3 MB/s eta 0:00:02
     ----------- ---------------------------- 2.0/7.2 MB 16.0 MB/s eta 0:00:01
     --------------------- ------------------ 3.9/7.2 MB 22.5 MB/s eta 0:00:01
     ------------------------------ --------- 5.6/7.2 MB 25.7 MB/s eta 0:00:01
     -------------------------------------- - 6.9/7.2 MB 26.1 MB/s eta 0:00:01
     ---------------------------------------- 7.2/7.2 MB 24.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  P


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
from athina.keys import AthinaApiKey, OpenAiApiKey

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

In [ ]:
!pip install athina[sandbox]


In [ ]:
!pip install ragas datasets evaluate


In [ ]:
!pip install "ragas[standard]" --upgrade


In [ ]:
!pip uninstall athina -y
!pip install -U athina

In [58]:
from athina.evals import RagasAnswerRelevancy

eval_model = "gpt-3.5-turbo"
RagasAnswerRelevancy(model=eval_model).run_batch(data=dataset).to_df()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"
ERROR:ragas.executor:Exception raised in Job[0]: AuthenticationError(Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope***-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}})
c:\Users\patel\AppData\Local\Programs\Python\Python310\lib\site-packages\athina\evals\ragas\ragas_evaluator.py:103: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(f"Invalid metric value: {metric_value}")
WARN: Invalid metric value: nan



Error logging dataset to Athina: AuthorizationError (Extra Info: please check your athina api key and try again)


,query,context,response,expected_response,display_name,failed,grade_reason,runtime,model
0,what are points on a mortgage,"[Discount points, also called mortgage points or simply points, are a form of pre-paid interest available in the United States when arranging a mortgage. One point equals one percent of the loan amount. By charging a borrower points, a lender effectively increases the yield on the loan above the amount of the stated interest rate. Borrowers can offer to pay a lender points as a method to reduce the interest rate on the loan, thus obtaining a lower monthly payment in exchange for this, points...","Points on a mortgage are a form of pre-paid interest that can be purchased to reduce the interest rate on the loan, resulting in a lower monthly payment for the borrower. It is equal to one percent of the loan amount and can be used to buy down the upfront payment.",None,Ragas Answer Relevancy,None,"A response is deemed relevant when it directly and appropriately addresses the original query. Importantly, our assessment of answer relevance does not consider factuality but instead penalizes cases where the response lacks completeness or contains redundant details",1859,gpt-3.5-turbo
